In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytimekr import pytimekr

In [2]:
df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
df.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [4]:
holiday = []
for year in range(2019, 2024):
    holiday += pytimekr.holidays(year=year)



def prerpocess(df, holiday=holiday):
    is_train = 'price(원/kg)' in df.columns
    is_holiday = []

    df.timestamp = pd.to_datetime(df.timestamp)
    df['year'] = df.timestamp.dt.year
    df['month'] = df.timestamp.dt.month
    df['day'] = df.timestamp.dt.day
    df['dow'] = df.timestamp.dt.day_of_week

    for time, dow in zip(df.timestamp, df.dow):
        if time in holiday or dow == 6:
            is_holiday.append(1)
        else:
            is_holiday.append(0)
    df['is_holiday'] = is_holiday

    df = pd.get_dummies(df, columns=['corporation', 'location'])
    
    if is_train:
        df.drop(['ID', 'timestamp', 'supply(kg)'], axis=1, inplace=True)
        df.rename(columns={'price(원/kg)' : 'price'}, inplace=True)
        df = df.loc[df['price'] != 0]
    else:
        df.drop(['ID', 'timestamp'], axis=1, inplace=True)
    
    return df

prerpocess(df).to_csv('./data/train_preproc2_.csv', index=False)
prerpocess(test_df).to_csv('./data/test_preproc2.csv', index=False)
